# Hey there! Here's more-or-less the steps you'll be taking to reduce our data, and, using those reduced data, extract some flux-calibrated lightcurves of WR 124! 

In [ ]:
#First, let's import some useful libraries
import numpy as np
from matplotlib import pyplot as plt
from adapt import *
from phot_tools import *
from glob import glob
import os
from astropy.io import fits

%matplotlib inline
#What do all of these libraries do?

## First things first, we need to reduce our data; that is, convert from the number that is stored in each pixel (which takes into account aaaaallllll of the optics and disturbances and quantum mechanics that are between the detector and the sky) to a number that we really hope corresponds to the number of photons that actually hit the detector.

In [ ]:
#First, point this notebook to the directory where your raw data are in (datadir), and 
#directories where you want the reduced master calibrations (caldir) and reduced science
#images (reddir) to go. You'll probably want to make the directories first. The trailing
#slash in the name is important (the code breaks if you don't give it the trailing slash...)

datadir = '/path/to/the/data/'
caldir = '/path/to/caldir/'
reddir = '/path/to/reddir/'

## Let's first measure the bias level in all of our images --- i.e., the little bit of signal that is inherent to every exposure --- by taking the median of a series of zero-second exposures.

In [ ]:
#Use glob to assemble lists of biases...
biaslist = glob(datadir+'string that glob can use to find biases. use * as a wildcard!')
print(biaslist) #this should be all biases...

In [ ]:
#Now create the master bias. You'll have to decide some parameters. Play with 
#overscan_fit_degree and see how it affects the output bias (which will now be
#in caldir/master_bias.fits). Do you notice any trend that affects each column of
#the bias? Try messing with overscan_fit_degree til it goes away. Overwrite determines 
#what will happen if the bias already exists. 
master_bias(biaslist=biaslist,overscan_fit_degree=?,caldir=caldir,overwrite=?)

In [ ]:
#Load up the bias and take a look at it. Does it look like there are any systematic 
#trends in the bias? Try messing with overscan_fit_degree. You can look at the image
#in ds9 or QFitsView, or just try loading it up in here!

## Now, because our detector isn't cooled to absolute zero, it adds a little bit of signal (called the dark current), which gets stronger with time. If we 'expose' the detector (without letting any light get to it) for the same amount of time as our actual images, we'll have an estimate of the dark current in those images. L

In [ ]:
#Ok, so we've made our bias... let's try making a dark! Because darks are dependent on exposure
#time, we'll have to make one dark for each exposure time. For now, just make a 10s dark
darklist = glob(datadir+'string that glob can use to find 10s darks.')
print(darklist) #did you only pick out the 10s darks?